In [1]:
import os
import csv
from datetime import timedelta
from django.apps import apps as django_apps
from edc_base.utils import get_utcnow
from flourish_child.helper_classes.utils import child_utils
from flourish_prn.action_items import MISSED_BIRTH_VISIT_ACTION
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
common_fields = ['weight_avail', 'weight_kg', 'length_avail',
                 'infant_length', 'head_circ_avail', 'head_circumference',
                 'apgar_score', 'apgar_score_min_1', 'apgar_score_min_5',
                 'apgar_score_min_10', 'congenital_anomalities', 'gestational_age']

In [3]:
def compare_instances(instance1, instance2):
    differences = {}
    for field in common_fields:
        value1 = getattr(instance1, field, None)
        value2 = getattr(instance2, field, None)
        if value1 != value2:
            differences.update(
                {f'{field}': f'missedbirthvisit: {value1}, visit {instance2.child_visit.visit_code}: {value2}'})
    return differences

In [4]:
def move_data_to_missed_birth_visit(birth_data):
    data_dict = {}
    for field in ['report_datetime', ] + common_fields:
        data_dict.update({f'{field}': getattr(birth_data, field, None)})
    child_birth = ChildBirth.objects.filter(
        subject_identifier=birth_data.subject_identifier).latest('report_datetime')
    data_dict.update({'infant_dob': child_birth.dob})
    return data_dict

In [5]:
from edc_action_item.site_action_items import site_action_items

def get_or_create_action_item(subject_identifier):
    action_cls = site_action_items.get(
        MissedBirthVisit.action_name)
    action_item_model_cls = action_cls.action_item_model_cls()
    _action_type = ActionType.objects.get(name=MISSED_BIRTH_VISIT_ACTION)

    obj, _ = action_item_model_cls.objects.get_or_create(
        subject_identifier=subject_identifier,
        action_type=_action_type)
    return obj

In [6]:
missed_del = ChildVisit.objects.filter(
    visit_code='2000D', reason='missed').values_list('subject_identifier', flat=True)

In [7]:
diff_checks = []
for idx in missed_del:
    try:
        missed_birth = MissedBirthVisit.objects.get(subject_identifier=idx)
    except MissedBirthVisit.DoesNotExist:
        birth_data = BirthData.objects.filter(child_visit__subject_identifier=idx)
        if birth_data.exists():
            print(idx, 'Birth Data without Missed Birth Visit, move data.')
            diff_checks.append({'subject_identifier': idx,
                                'discrepant_field': None,
                                'issue': 'Birth Data without Missed Birth Visit'})
            qs_data = move_data_to_missed_birth_visit(birth_data.first())
            action_item = get_or_create_action_item(idx)
            qs_data.update({
                'subject_identifier': idx,
                'action_identifier': action_item.action_identifier})

            try:
                MissedBirthVisit.objects.create(**qs_data)
            except Exception:
                raise
            else:
                birth_data.delete()
    else:
        birth_data = BirthData.objects.filter(child_visit__subject_identifier=idx)
        if not birth_data.exists():
            continue
        else:
            _differs = compare_instances(missed_birth, birth_data.first())
            if _differs:
                for affected_field, discrepancy in _differs.items():
                    diff_checks.append({
                        'subject_identifier': idx,
                        'discrepant_field': affected_field,
                        'issue': discrepancy})
            else:
                print(idx, 'has no differences')
                diff_checks.append({'subject_identifier': idx,
                                    'discrepant_field': None,
                                    'issue': 'Duplicate data for Birth Data and Missed Birth Visit, remove birth data.'})
                birth_data.delete()

B142-040990430-6-10 Birth Data without Missed Birth Visit, move data.
B142-040990481-9-10 Birth Data without Missed Birth Visit, move data.
B142-040990486-8-60 has no differences
B142-040990508-9-10 Birth Data without Missed Birth Visit, move data.
B142-040990556-8-10 Birth Data without Missed Birth Visit, move data.
B142-040990577-4-10 Birth Data without Missed Birth Visit, move data.
B142-040990609-5-10 Birth Data without Missed Birth Visit, move data.
B142-040990615-2-10 Birth Data without Missed Birth Visit, move data.
B142-040990634-3-10 Birth Data without Missed Birth Visit, move data.
B142-040990640-0-10 Birth Data without Missed Birth Visit, move data.
B142-040990676-4-10 Birth Data without Missed Birth Visit, move data.
B142-040990677-2-10 Birth Data without Missed Birth Visit, move data.
B142-040990690-5-10 Birth Data without Missed Birth Visit, move data.
B142-040990691-3-10 Birth Data without Missed Birth Visit, move data.
B142-040990704-4-10 Birth Data without Missed Birth

In [8]:
diff_checks

[{'subject_identifier': 'B142-040990369-6-60',
  'discrepant_field': 'weight_kg',
  'issue': 'missedbirthvisit: 3.16, visit 2001: 3.10'},
 {'subject_identifier': 'B142-040990369-6-60',
  'discrepant_field': 'head_circumference',
  'issue': 'missedbirthvisit: 37.00, visit 2001: 32.00'},
 {'subject_identifier': 'B142-040990430-6-10',
  'discrepant_field': None,
  'issue': 'Birth Data without Missed Birth Visit'},
 {'subject_identifier': 'B142-040990481-9-10',
  'discrepant_field': None,
  'issue': 'Birth Data without Missed Birth Visit'},
 {'subject_identifier': 'B142-040990486-8-60',
  'discrepant_field': None,
  'issue': 'Duplicate data for Birth Data and Missed Birth Visit, remove birth data.'},
 {'subject_identifier': 'B142-040990508-9-10',
  'discrepant_field': None,
  'issue': 'Birth Data without Missed Birth Visit'},
 {'subject_identifier': 'B142-040990556-8-10',
  'discrepant_field': None,
  'issue': 'Birth Data without Missed Birth Visit'},
 {'subject_identifier': 'B142-04099057

In [9]:
import csv

keys = diff_checks[0].keys()
with open('child_birth_data_queries.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=keys)
    writer.writeheader()
    writer.writerows(diff_checks)